# Моделирование изменения балансов студентов

<h3><b>Задача — смоделировать изменение балансов студентов.</b></h3>

<p><b>Баланс</b> — это количество уроков, которое есть у каждого студента.</p>
<p>Чтобы проверить, всё ли в порядке с нашими данными, составьте список гипотез и вопросов.
Нам важно понимать:</p>
<ul>
<li>сколько всего уроков было на балансе всех учеников за каждый календарный день;</li>
<li>как это количество менялось под влиянием транзакций (оплат, начислений, корректирующих списаний) и уроков (списаний с баланса по мере прохождения уроков).</li></ul>
<p>Также мы хотим создать таблицу, где будут балансы каждого студента за каждый день.</p>

<h3><b>Описание БД</b></h3>

<b><font color=#000066> SKYENG_DB </font></b>

<b><font color=#000066>classes</font></b></br>
<i>Витрина с уроками</i>
<ul>
<li>user_id - уникальный идентификатор юзера</li>
<li>id_class - уникальный идентификатор урока</li>
<li>class_start_datetime - время начала урока</li>
<li>class_end_datetime - время конца урока</li>
<li>class_removed_datetime - время удаления записи о данном уроке</li>
<li>id_teacher - уникальный идентификатор учителя</li>
<li>class_status - статус урока (успешно проведен / отменен и тд)</li>
<li>class_status_datetime - время проставления статуса по уроку</li></ul>
<b><font color=#000066>payments</font></b></br>
<i>Витрина с платежами по урокам</i>
<ul>
<li>user_id - уникальный идентификатор юзера</li>
<li>id_transaction - уникальный идентификатор транзакции</li>
<li>operation_name - название проведенной операции</li>
<li>status_name - статус проведенной операции (исполнена / не исполнена и тд)</li>
<li>classes - количество оплаченных уроков</li>
<li>payment_amount - выплаченная сумма</li>
<li>transaction_datetime - время проведения операции</li>

<p><b><font color=#000066>students</font></b></p>
<i>Витрина со списком студентов</i>
<ul>
<li>user_id - уникальный идентификатор юзера</li>
<li>student_sex - пол юзера</li>
<li>geo_cluster - географическая агрегация</li>
<li>country_name - короткое название страны</li>
<li>region_name - название региона</li>
<li>email_domain - домен электронной почты</li></ul>
<b><font color=#000066>teachers</font></b></br>
<i>Витрина со списком учителей</i>
<ul>
<li>id_teacher - уникальный идентификатор учителя</li>
<li>age - возраст</li>
<li>city - город проживания учителя</li>
<li>department - направление, в котором работает учитель</li>
<li>max_teaching_level - название уровня языка у преподавателя</li>
<li>id_teaching_level - уникальный идентификатор уровня языка у преподавателя</li>
<li>language_group - основной язык преподавателя</li></ul>

## Код

In [ ]:
with first_payments as -- определяем дату первой успешной транзакции для каждого студента
    (
    select distinct user_id
        , min(transaction_datetime::date) first_payment 
    from skyeng_db.payments
    where status_name = 'success'
    group by user_id
    )
, all_dates as -- определяем уникальные даты занятий в 2016 году
    (
    select distinct date_trunc('day', class_start_datetime) dt 
    from skyeng_db.classes
    where date_trunc ('year', class_start_datetime) = '2016.01.01'
    ) 
, payments_by_dates as -- находим изменения балансов, связанных с успешными транзакциями
    (
    select user_id
        , date_trunc('day', transaction_datetime) as dt 
        , sum(classes) as transaction_balance_change
    from skyeng_db.payments 
        where status_name = 'success'
    group by user_id
        , dt
    )  
, all_dates_by_user as -- определяем даты для сбора баланса
    (
    select a.user_id
        , b.dt
    from first_payments a
        join all_dates b
            on a.first_payment <= b.dt
    order by user_id, dt
    )
, classes_by_dates as -- находим изменения балансов, связанных с прохождением уроков
    (
    select user_id
        , date_trunc('day', class_start_datetime) class_date
        , count(id_class)*(-1) classes 
    from skyeng_db.classes
    where class_status in ('success', 'failed_by_student')
            and class_type != 'trial'
    group by user_id
        ,class_date
    )
, payments_by_dates_cumsum as -- находим изменения балансов студентов, связанные с покупкой уроков (добавляем уроки)
    (
    select a.user_id
        , a.dt
        , b.transaction_balance_change
        , sum(coalesce(b.transaction_balance_change,0)) over (partition by a.user_id order by a.dt) transaction_balance_change_cs
    from all_dates_by_user a
        left join payments_by_dates b
            on a.user_id = b.user_id and a.dt = b.dt
            )
, classes_by_dates_dates_cumsum as -- находим изменения балансов студентов, связанные с прохождением уроков (списываем уроки)
    (
    select a.user_id
        , a.dt
        , b.classes
        , sum(coalesce(b.classes,0)) over (partition by a.user_id order by a.dt) classes_cs
    from all_dates_by_user a
        left join classes_by_dates b
            on a.user_id = b.user_id and a.dt = b.class_date
    )
 , balance as -- определяем состояние баланса каждого пользователя за каждый "прожитый" день
    (
    select a.user_id
        , a.dt
        , a.transaction_balance_change -- сколько уроков куплено за день
        , a.transaction_balance_change_cs -- сколько уроков куплено всего за все время
        , b.classes -- сколько уроков пройдено за день
        , b.classes_cs -- сколько уроков пройдено за все время
        , b.classes_cs + a.transaction_balance_change_cs as balance -- количество оплаченных, но непройденных уроков
    from payments_by_dates_cumsum  a
        join classes_by_dates_dates_cumsum b
            on a.user_id = b.user_id
                and a.dt = b.dt
    order by a.user_id, a.dt
    -- limit 2000
    )
select dt -- получаем суммарные данные баланса по всем пользователям за 2016 год по дням
    , sum(transaction_balance_change) new_lessons -- сколько уроков куплено за день
    , sum(transaction_balance_change_cs) lessons_on_balance -- сколько уроков куплено за все время к этому дню
    , sum (classes) new_classes -- сколько уроков проедено за день
    , sum (classes_cs) lessons_off_ballance -- сколько уроков пройдено за все время к этому дню
    , sum (balance) actual_balance -- количество оплаченных, но не пройденных уроков
from balance
group by dt
order by dt